# PDF

[Portable Document Format (PDF)](https://en.wikipedia.org/wiki/PDF), ISO 32000으로 표준화된 파일 형식은 Adobe가 1992년에 문서를 제시하기 위해 개발했으며, 이는 응용 소프트웨어, 하드웨어 및 운영 시스템에 독립적인 방식으로 텍스트 서식 및 이미지를 포함합니다.

이 가이드는 `PDF` 문서를 LangChain [Document](https://api.python.langchain.com/en/latest/documents/langchain_core.documents.base.Document.html#langchain_core.documents.base.Document) 형식으로 로드하는 방법을 다룹니다. 이 형식은 다운스트림에서 사용됩니다.

LangChain은 다양한 PDF 파서와 통합됩니다. 일부는 간단하고 상대적으로 저수준이며, 다른 일부는 OCR 및 이미지 처리를 지원하거나 고급 문서 레이아웃 분석을 수행합니다. 올바른 선택은 사용자의 애플리케이션에 따라 달라집니다. 아래에서 가능성을 열거합니다.

In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

## 실습에 활용한 문서

소프트웨어정책연구소(SPRi) - 2023년 12월호

- 저자: 유재흥(AI정책연구실 책임연구원), 이지수(AI정책연구실 위촉연구원)
- 링크: https://spri.kr/posts/view/23669
- 파일명: `SPRI_AI_Brief_2023년12월호_F.pdf`

**참고**: 위의 파일은 `data` 폴더 내에 다운로드 받으세요

In [13]:
DATA_PATH = "./data/SPRI_AI_Brief_2023년12월호_F.pdf"

## PyPDF

여기에서는 `pypdf`를 사용하여 PDF를 문서 배열로 로드하며, 각 문서는 `page` 번호와 함께 페이지 내용 및 메타데이터를 포함합니다.

In [ ]:
# 설치
# !pip install -qU pypdf

In [14]:
from langchain_community.document_loaders import PyPDFLoader

file_path = DATA_PATH
# 파일 경로 설정
loader = PyPDFLoader(file_path)
# PDF 로더 초기화
pages = loader.load_and_split()
# 페이지 로드 및 분할

pages[0]
# 첫 번째 페이지 출력

Document(page_content='2023 년 12월호', metadata={'source': './data/SPRI_AI_Brief_2023년12월호_F.pdf', 'page': 0})

이 접근 방식의 장점은 페이지 번호로 문서를 검색할 수 있다는 것입니다.

### PDF를 통한 벡터 검색

LangChain **Document** 객체에 PDF를 로드한 후에는 일반적인 방법으로 색인을 생성할 수 있습니다(예: RAG 애플리케이션):

In [15]:
# !pip install -qU faiss-cpu
# CUDA GPU 지원을 위한 `pip install faiss-gpu` 사용

In [4]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

# FAISS 인덱스 생성
faiss_index = FAISS.from_documents(pages, OpenAIEmbeddings())
# 유사성 검색 실행
docs = faiss_index.similarity_search("삼성전자가 만든 생성형 AI", k=2)
for doc in docs:
    # 페이지 번호 및 내용 출력
    print(str(doc.metadata["page"]) + ":", doc.page_content[:300])

12: SPRi AI Brief |  
2023-12 월호
10삼성전자 , 자체 개발 생성 AI ‘삼성 가우스 ’ 공개
n삼성전자가 온디바이스에서 작동 가능하며 언어, 코드, 이미지의 3개 모델로 구성된 자체 개발 생성 
AI 모델 ‘삼성 가우스 ’를 공개
n삼성전자는 삼성 가우스를 다양한 제품에 단계적으로 탑재할 계획으로 , 온디바이스 작동이 가능한 
삼성 가우스는 외부로 사용자 정보가 유출될 위험이 없다는 장점을 보유KEY Contents
£언어, 코드, 이미지의 3개 모델로 구성된 삼성 가우스 , 온디바이스 작동 지원
n삼성전자가 2
15: 1. 정책/법제  2. 기업/산업 3. 기술/연구  4. 인력/교육
빌 게이츠 , AI 에이전트로 인한 컴퓨터 사용의 패러다임 변화 전망
n빌 게이츠가 5년 내 일상언어로 모든 작업을 처리할 수 있는 AI 에이전트가 보급되며 컴퓨터를 
사용하는 방식이 완전히 바뀔 것으로 예상
n에이전트의 보급은 컴퓨터 분야를 넘어 산업 전 영역에 영향을 미칠 전망으로 특히 의료와 
교육, 생산성 , 엔터테인먼트 ·쇼핑 영역에서 고가로 제공되던 서비스가 대중화될 전망KEY Contents
£5년 내 기기에 일상언어로 말하기만 하면 되는 AI 에이전트


### OCR

일부 PDF에는 스캔된 문서나 그림 내에 텍스트 이미지가 포함되어 있습니다. `rapidocr-onnxruntime` 패키지를 사용하여 이미지에서 텍스트를 추출할 수도 있습니다.

In [5]:
# 설치
# !pip install -qU rapidocr-onnxruntime

In [12]:
# PDF 로더 초기화, 이미지 추출 옵션 활성화
loader = PyPDFLoader("https://arxiv.org/pdf/2103.15348.pdf", extract_images=True)
# PDF 페이지 로드
pages = loader.load()
# 다섯 번째 페이지 내용 접근
pages[4].page_content

'LayoutParser : A Uniﬁed Toolkit for DL-Based DIA 5\nTable 1: Current layout detection models in the LayoutParser model zoo\nDataset Base Model1Large Model Notes\nPubLayNet [38] F / M M Layouts of modern scientiﬁc documents\nPRImA [3] M - Layouts of scanned modern magazines and scientiﬁc reports\nNewspaper [17] F - Layouts of scanned US newspapers from the 20th century\nTableBank [18] F F Table region on modern scientiﬁc and business document\nHJDataset [31] F / M - Layouts of history Japanese documents\n1For each dataset, we train several models of diﬀerent sizes for diﬀerent needs (the trade-oﬀ between accuracy\nvs. computational cost). For “base model” and “large model”, we refer to using the ResNet 50 or ResNet 101\nbackbones [ 13], respectively. One can train models of diﬀerent architectures, like Faster R-CNN [ 28] (F) and Mask\nR-CNN [ 12] (M). For example, an F in the Large Model column indicates it has a Faster R-CNN model trained\nusing the ResNet 101 backbone. The platform i

**PyMuPDF** 는 속도 최적화가 되어 있으며, PDF 및 해당 페이지에 대한 자세한 메타데이터를 포함하고 있습니다. 페이지 당 하나의 문서를 반환합니다:

In [ ]:
# 설치
# !pip install -qU pymupdf

In [16]:
from langchain_community.document_loaders import PyMuPDFLoader

# PyMuPDF 로더 인스턴스 생성
loader = PyMuPDFLoader(DATA_PATH)
# 문서 로드
data = loader.load()
# 첫 번째 페이지 데이터 접근
data[0]

Document(page_content='2023년 12월호\n', metadata={'source': './data/SPRI_AI_Brief_2023년12월호_F.pdf', 'file_path': './data/SPRI_AI_Brief_2023년12월호_F.pdf', 'page': 0, 'total_pages': 23, 'format': 'PDF 1.4', 'title': '', 'author': 'dj', 'subject': '', 'keywords': '', 'creator': 'Hwp 2018 10.0.0.13462', 'producer': 'Hancom PDF 1.3.0.542', 'creationDate': "D:20231208132838+09'00'", 'modDate': "D:20231208132838+09'00'", 'trapped': ''})

`load` 호출 시 [PyMuPDF documentation](https://pymupdf.readthedocs.io/en/latest/app1.html#plain-text/)에서 제공하는 옵션들을 키워드 인자로 전달할 수 있으며, 이는 `get_text()` 호출에 전달됩니다.

In [ ]:
from langchain_community.document_loaders import MathpixPDFLoader

file_path = DATA_PATH
# 파일 경로 설정
loader = MathpixPDFLoader(file_path)
# Mathpix PDF 로더 인스턴스 생성
data = loader.load()
# 데이터 로드
data[0]
# 첫 번째 데이터 항목 접근

[Unstructured](https://unstructured-io.github.io/unstructured/)는 Markdown이나 PDF와 같은 비구조화된 또는 반구조화된 파일 형식을 다루기 위한 공통 인터페이스를 지원합니다. LangChain의 [UnstructuredPDFLoader](https://api.python.langchain.com/en/latest/document_loaders/langchain_community.document_loaders.pdf.UnstructuredPDFLoader.html)는 Unstructured와 통합되어 PDF 문서를 LangChain [Document](https://api.python.langchain.com/en/latest/documents/langchain_core.documents.base.Document.html) 객체로 파싱합니다.

시스템 요구 사항 설치에 대한 자세한 정보는 [이 페이지](/docs/integrations/providers/unstructured/)를 참조하십시오.

In [18]:
# 설치
# !pip install -qU unstructured

In [19]:
from langchain_community.document_loaders import UnstructuredPDFLoader

# 비구조화된 PDF 로더 인스턴스 생성
loader = UnstructuredPDFLoader(DATA_PATH)
# 데이터 로드
data = loader.load()
# 첫 번째 데이터 항목 접근
data[0]

Document(page_content='2023년 12월호\n\n2023년 12월호\n\nⅠ. 인공지능 산업 동향 브리프\n\n1. 정책/법제\n\n▹ 미국, 안전하고 신뢰할 수 있는 AI 개발과 사용에 관한 행정명령 발표 ························· 1\n\n▹ G7, 히로시마 AI 프로세스를 통해 AI 기업 대상 국제 행동강령에 합의 ··························· 2\n\n▹ 영국 AI 안전성 정상회의에 참가한 28개국, AI 위험에 공동 대응 선언 ··························· 3\n\n▹ 미국 법원, 예술가들이 생성 AI 기업에 제기한 저작권 소송 기각 ····································· 4\n\n▹ 미국 연방거래위원회, 저작권청에 소비자 보호와 경쟁 측면의 AI 의견서 제출 ················· 5\n\n▹ EU AI 법 3자 협상, 기반모델 규제 관련 견해차로 난항 ··················································· 6\n\n2. 기업/산업\n\n▹ 미국 프런티어 모델 포럼, 1,000만 달러 규모의 AI 안전 기금 조성 ································ 7\n\n▹ 코히어, 데이터 투명성 확보를 위한 데이터 출처 탐색기 공개 ······································· 8\n\n▹ 알리바바 클라우드, 최신 LLM ‘통이치엔원 2.0’ 공개 ······················································ 9\n\n▹ 삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개 ··························································· 10\n\n▹ 구글, 앤스로픽에 20억 달러 투자로 생성 AI 협력 강화 ············································

내부적으로 비정형에서는 텍스트 청크마다 서로 다른 "**요소**"를 만듭니다. 기본적으로 이들은 결합되어 있지만 `mode="elements"`를 지정하여 쉽게 분리할 수 있습니다.

In [20]:
# 파일 경로 설정
loader = UnstructuredPDFLoader(DATA_PATH, mode="elements")
# 비구조화된 PDF 로더 초기화

data = loader.load()
# 데이터 로드
data[0]
# 첫 번째 데이터 요소 접근

Document(page_content='2023년 12월호', metadata={'source': './data/SPRI_AI_Brief_2023년12월호_F.pdf', 'coordinates': {'points': ((256.579467, 282.444348), (256.579467, 303.42387300000007), (355.4236898438, 303.42387300000007), (355.4236898438, 282.444348)), 'system': 'PixelSpace', 'layout_width': 612, 'layout_height': 858}, 'file_directory': './data', 'filename': 'SPRI_AI_Brief_2023년12월호_F.pdf', 'languages': ['eng'], 'last_modified': '2024-03-11T18:59:07', 'page_number': 1, 'filetype': 'application/pdf', 'category': 'UncategorizedText'})

이 특정 문서에 대한 전체 요소 유형 집합을 참조하세요

In [21]:
set(doc.metadata["category"] for doc in data)  # 데이터 카테고리 추출

{'ListItem', 'NarrativeText', 'Title', 'UncategorizedText'}

## OnlinePDFLoader

이는 온라인 PDF를 문서 형식으로 로드하는 방법을 다룹니다. 이 방법은 https://open.umn.edu/opentextbooks/textbooks/ 및 https://arxiv.org/archive/ 와 같은 다양한 온라인 PDF 사이트에 사용할 수 있습니다.

참고: 다른 모든 PDF 로더도 원격 PDF를 가져오는 데 사용할 수 있지만, `OnlinePDFLoader`는 레거시 함수이며 `UnstructuredPDFLoader`와 특별히 함께 작동합니다.

In [22]:
from langchain_community.document_loaders import OnlinePDFLoader

# 온라인 PDF 로더 인스턴스 생성
loader = OnlinePDFLoader("https://arxiv.org/pdf/2302.03803.pdf")
# 데이터 로드
data = loader.load()
# 첫 번째 데이터 요소 접근
data[0]

Document(page_content='3 2 0 2\n\nb e F 7\n\n]\n\nG A . h t a m\n\n[\n\n1 v 3 0 8 3 0 . 2 0 3 2 : v i X r a\n\nA WEAK (k, k)-LEFSCHETZ THEOREM FOR PROJECTIVE TORIC ORBIFOLDS\n\nWilliam D. Montoya\n\nInstituto de Matem´atica, Estat´ıstica e Computa¸c˜ao Cient´ıﬁca, Universidade Estadual de Campinas (UNICAMP),\n\nRua S´ergio Buarque de Holanda 651, 13083-859, Campinas, SP, Brazil\n\nFebruary 9, 2023\n\nAbstract\n\nFirstly we show a generalization of the (1, 1)-Lefschetz theorem for projective toric orbifolds and secondly we prove that on 2k-dimensional quasi-smooth hyper- surfaces coming from quasi-smooth intersection surfaces, under the Cayley trick, every rational (k, k)-cohomology class is algebraic, i.e., the Hodge conjecture holds on them.\n\n1\n\nIntroduction\n\nIn [3] we proved that, under suitable conditions, on a very general codimension s quasi- smooth intersection subvariety X in a projective toric orbifold Pd Σ with d + s = 2(k + 1) the Hodge conjecture holds, that is, every 

## PyPDFium2

In [23]:
from langchain_community.document_loaders import PyPDFium2Loader

# 파일 경로 설정
loader = PyPDFium2Loader(DATA_PATH)
# PyPDFium2 로더 인스턴스 생성
data = loader.load()
# 데이터 로드
data[0]
# 첫 번째 데이터 항목 접근

Document(page_content='2023년 12월호\n', metadata={'source': './data/SPRI_AI_Brief_2023년12월호_F.pdf', 'page': 0})

## PDFMiner

In [24]:
from langchain_community.document_loaders import PDFMinerLoader

# PDF 파일 경로
loader = PDFMinerLoader(DATA_PATH)
# PDFMiner 로더 인스턴스 생성
data = loader.load()
# 데이터 로드
data[0]
# 첫 번째 데이터 항목 접근

Document(page_content='2023년  12월호\n\n\x0c2023년  12월호\n\nⅠ.  인공지능  산업  동향  브리프\n\n  1.  정책/법제 \n\n      ▹  미국,  안전하고  신뢰할  수  있는  AI  개발과  사용에  관한  행정명령  발표    ························· 1\n\n      ▹  G7,  히로시마  AI  프로세스를  통해  AI  기업  대상  국제  행동강령에  합의 ··························· 2\n\n      ▹  영국  AI  안전성  정상회의에  참가한  28개국,  AI  위험에  공동  대응  선언 ··························· 3\n\n      ▹  미국  법원,  예술가들이  생성  AI  기업에  제기한  저작권  소송  기각 ····································· 4\n\n      ▹  미국  연방거래위원회,  저작권청에  소비자  보호와  경쟁  측면의  AI  의견서  제출 ················· 5\n\n      ▹  EU  AI  법  3자  협상,  기반모델  규제  관련  견해차로  난항 ··················································· 6\n\n  2.  기업/산업 \n\n      ▹  미국  프런티어  모델  포럼,  1,000만  달러  규모의  AI  안전  기금  조성 ································ 7\n\n      ▹  코히어,  데이터  투명성  확보를  위한  데이터  출처  탐색기  공개    ······································· 8\n\n      ▹  알리바바  클라우드,  최신  LLM  ‘통이치엔원  2.0’  공개  ······················································ 9\n\n      ▹ 

**PDFMiner**를 사용하여 HTML 텍스트 생성

이 방법은 출력된 HTML 콘텐츠를 `BeautifulSoup`을 통해 파싱함으로써 글꼴 크기, 페이지 번호, PDF 헤더/푸터 등에 대한 보다 구조화되고 풍부한 정보를 얻을 수 있게 하여 텍스트를 의미론적으로 섹션으로 분할하는 데 도움이 될 수 있습니다.

In [25]:
from langchain_community.document_loaders import PDFMinerPDFasHTMLLoader

# 파일 경로 설정
loader = PDFMinerPDFasHTMLLoader(DATA_PATH)
# PDFMinerPDFasHTMLLoader 인스턴스 생성
docs = loader.load()
# 문서 로드
docs[0]
# 첫 번째 문서 출력

Document(page_content='<html><head>\n<meta http-equiv="Content-Type" content="text/html">\n</head><body>\n<span style="position:absolute; border: gray 1px solid; left:0px; top:50px; width:612px; height:858px;"></span>\n<div style="position:absolute; top:50px;"><a name="1">Page 1</a></div>\n<div style="position:absolute; border: textbox 1px solid; writing-mode:lr-tb; left:256px; top:332px; width:98px; height:20px;"><span style="font-family: KoPubDotumBold; font-size:14px">2023년  </span><span style="font-family: KoPubDotumBold; font-size:20px">12</span><span style="font-family: KoPubDotumBold; font-size:14px">월호\n<br></span></div><div style="position:absolute; border: figure 1px solid; writing-mode:False; left:0px; top:50px; width:612px; height:858px;"></div><span style="position:absolute; border: gray 1px solid; left:0px; top:958px; width:612px; height:858px;"></span>\n<div style="position:absolute; top:958px;"><a name="2">Page 2</a></div>\n<div style="position:absolute; border: textbox

In [26]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(docs[0].page_content, "html.parser")  # HTML 파서 초기화
content = soup.find_all("div")  # 모든 div 태그 검색

In [27]:
import re

cur_fs = None
cur_text = ""
snippets = []  # 동일한 글꼴 크기의 모든 스니펫 수집
for c in content:
    sp = c.find("span")
    if not sp:
        continue
    st = sp.get("style")
    if not st:
        continue
    fs = re.findall("font-size:(\d+)px", st)
    if not fs:
        continue
    fs = int(fs[0])
    if not cur_fs:
        cur_fs = fs
    if fs == cur_fs:
        cur_text += c.text
    else:
        snippets.append((cur_text, cur_fs))
        cur_fs = fs
        cur_text = c.text
snippets.append((cur_text, cur_fs))
# 중복 스니펫 제거 전략 추가 가능성 (PDF의 헤더/푸터가 여러 페이지에 걸쳐 나타나므로 중복 발견 시 중복 정보로 간주 가능)

In [28]:
from langchain_core.documents import Document

cur_idx = -1
semantic_snippets = []
# 제목 가정: 높은 글꼴 크기
for s in snippets:
    # 새 제목 판별: 현재 스니펫 글꼴 > 이전 제목 글꼴
    if (
        not semantic_snippets
        or s[1] > semantic_snippets[cur_idx].metadata["heading_font"]
    ):
        metadata = {"heading": s[0], "content_font": 0, "heading_font": s[1]}
        metadata.update(docs[0].metadata)
        semantic_snippets.append(Document(page_content="", metadata=metadata))
        cur_idx += 1
        continue

    # 동일 섹션 내용 판별: 현재 스니펫 글꼴 <= 이전 내용 글꼴
    if (
        not semantic_snippets[cur_idx].metadata["content_font"]
        or s[1] <= semantic_snippets[cur_idx].metadata["content_font"]
    ):
        semantic_snippets[cur_idx].page_content += s[0]
        semantic_snippets[cur_idx].metadata["content_font"] = max(
            s[1], semantic_snippets[cur_idx].metadata["content_font"]
        )
        continue

    # 새 섹션 생성 조건: 현재 스니펫 글꼴 > 이전 내용 글꼴, 이전 제목 글꼴 미만
    metadata = {"heading": s[0], "content_font": 0, "heading_font": s[1]}
    metadata.update(docs[0].metadata)
    semantic_snippets.append(Document(page_content="", metadata=metadata))
    cur_idx += 1

print(semantic_snippets[4])

page_content='KEY Contents\nn 미국 바이든 대통령이 ‘안전하고 신뢰할 수 있는 AI 개발과 사용에 관한 행정명령’에 서명하고 \n광범위한  행정  조치를  명시\nn 행정명령은 △AI의 안전과 보안 기준 마련 △개인정보보호 △형평성과 시민권 향상 △소비자 \n보호  △노동자  지원  △혁신과  경쟁  촉진  △국제협력을  골자로  함\n' metadata={'heading': '미국,  안전하고  신뢰할  수  있는  AI  개발과  사용에  관한  행정명령  발표 \n', 'content_font': 12, 'heading_font': 15, 'source': './data/SPRI_AI_Brief_2023년12월호_F.pdf'}


## PyPDF 디렉토리

디렉토리에서 PDF를 로드하세요

In [29]:
from langchain_community.document_loaders import PyPDFDirectoryLoader

# 디렉토리 경로
loader = PyPDFDirectoryLoader("data/")

# 문서 로드
docs = loader.load()

# 첫 번째 문서 데이터 접근
data[0]

Document(page_content='2023년  12월호\n\n\x0c2023년  12월호\n\nⅠ.  인공지능  산업  동향  브리프\n\n  1.  정책/법제 \n\n      ▹  미국,  안전하고  신뢰할  수  있는  AI  개발과  사용에  관한  행정명령  발표    ························· 1\n\n      ▹  G7,  히로시마  AI  프로세스를  통해  AI  기업  대상  국제  행동강령에  합의 ··························· 2\n\n      ▹  영국  AI  안전성  정상회의에  참가한  28개국,  AI  위험에  공동  대응  선언 ··························· 3\n\n      ▹  미국  법원,  예술가들이  생성  AI  기업에  제기한  저작권  소송  기각 ····································· 4\n\n      ▹  미국  연방거래위원회,  저작권청에  소비자  보호와  경쟁  측면의  AI  의견서  제출 ················· 5\n\n      ▹  EU  AI  법  3자  협상,  기반모델  규제  관련  견해차로  난항 ··················································· 6\n\n  2.  기업/산업 \n\n      ▹  미국  프런티어  모델  포럼,  1,000만  달러  규모의  AI  안전  기금  조성 ································ 7\n\n      ▹  코히어,  데이터  투명성  확보를  위한  데이터  출처  탐색기  공개    ······································· 8\n\n      ▹  알리바바  클라우드,  최신  LLM  ‘통이치엔원  2.0’  공개  ······················································ 9\n\n      ▹ 

## PDFPlumber

PyMuPDF와 마찬가지로, 출력 문서는 PDF와 그 페이지에 대한 자세한 메타데이터를 포함하며, 페이지 당 하나의 문서를 반환합니다.

In [33]:
from langchain_community.document_loaders import PDFPlumberLoader

# PDF 문서 로더 인스턴스 생성
loader = PDFPlumberLoader(DATA_PATH)

# 문서 로딩
data = loader.load()
# 첫 번째 문서 데이터 접근
data[0]

Document(page_content='12\n2023년 월호\n', metadata={'source': './data/SPRI_AI_Brief_2023년12월호_F.pdf', 'file_path': './data/SPRI_AI_Brief_2023년12월호_F.pdf', 'page': 0, 'total_pages': 23, 'Author': 'dj', 'Creator': 'Hwp 2018 10.0.0.13462', 'Producer': 'Hancom PDF 1.3.0.542', 'CreationDate': "D:20231208132838+09'00'", 'ModDate': "D:20231208132838+09'00'", 'PDFVersion': '1.4'})

## AmazonTextractPDFParser

**AmazonTextractPDFLoader**는 [Amazon Textract Service](https://aws.amazon.com/textract/)를 호출하여 PDF를 문서 구조로 변환합니다. 현재는 순수 OCR을 수행하며, 수요에 따라 레이아웃 지원과 같은 추가 기능이 계획되어 있습니다. 단일 및 다중 페이지 문서를 최대 3000페이지 및 512MB 크기까지 지원합니다.

호출이 성공적으로 이루어지려면 [AWS CLI](https://docs.aws.amazon.com/cli/latest/userguide/cli-chap-configure.html) 요구 사항과 유사한 AWS 계정이 필요합니다.

AWS 구성 외에도 JPEG, PNG 및 TIFF 및 비네이티브 PDF 형식을 지원하면서 다른 PDF 로더와 매우 유사합니다.

In [ ]:
from langchain_community.document_loaders import AmazonTextractPDFLoader

# Amazon Textract PDF 로더 인스턴스 생성
loader = AmazonTextractPDFLoader("example_data/alejandro_rosalez_sample-small.jpeg")

# 문서 로드
documents = loader.load()

# 첫 번째 문서 접근
documents[0]

[Azure AI Document Intelligence](https://aka.ms/doc-intelligence)(이전 명칭: `Azure Form Recognizer`)는 글씨(손글씨 포함), 표, 문서 구조(예: 제목, 절 제목 등) 및 키-값 쌍을 디지털 또는 스캔된 PDF, 이미지, Office 및 HTML 파일에서 추출하는 기계 학습 기반 서비스입니다. Document Intelligence는 `PDF`, `JPEG/JPG`, `PNG`, `BMP`, `TIFF`, `HEIF`, `DOCX`, `XLSX`, `PPTX`, `HTML`을 지원합니다.

이 [현재 구현](https://aka.ms/di-langchain)은 `Document Intelligence`를 사용하여 페이지별로 내용을 통합하고 LangChain 문서로 변환할 수 있습니다. 기본 출력 형식은 마크다운이며, `MarkdownHeaderTextSplitter`와 연결하여 의미론적 문서 분할을 쉽게 할 수 있습니다. 또한 `mode="single"` 또는 `mode="page"`를 사용하여 단일 페이지 또는 페이지별로 분할된 순수 텍스트를 반환할 수 있습니다.

### Prerequisite

세 개의 미리보기 지역 중 하나인 **East US**, **West US2**, **West Europe**에 Azure AI Document Intelligence 리소스가 필요합니다. 없는 경우 [이 문서](https://learn.microsoft.com/azure/ai-services/document-intelligence/create-document-intelligence-resource?view=doc-intel-4.0.0)를 참조하여 생성하세요. 로더에 `<endpoint>`와 `<key>`를 매개변수로 전달하게 됩니다.

In [ ]:
# 설치
# !pip install -qU azure-ai-documentintelligence

In [ ]:
from langchain_community.document_loaders import AzureAIDocumentIntelligenceLoader

file_path = "<filepath>"
endpoint = "<endpoint>"
key = "<key>"
# Azure AI 문서 인텔리전스 로더 초기화
loader = AzureAIDocumentIntelligenceLoader(
    api_endpoint=endpoint, api_key=key, file_path=file_path, api_model="prebuilt-layout"
)

# 문서 로드
documents = loader.load()

# 첫 번째 문서 접근
documents[0]